# Trajectory Tutorial

Learn how to work with `Trajectory` objects in MolPy! Trajectories represent sequences of molecular frames - perfect for analyzing simulation data.


## What is a Trajectory?

A `Trajectory` is a sequence of `Frame` objects representing time evolution:

- **Lazy Loading**: Frames are loaded on-demand, not all at once
- **Memory Efficient**: Supports memory-mapped file reading
- **Iterable**: Loop through frames like any Python sequence
- **Sliceable**: Get subsets of frames with slicing
- **Mappable**: Apply functions to all frames

Perfect for analyzing large MD simulations!


In [ ]:
import numpy as np

import molpy as mp
from molpy.core.trajectory import FrameGenerator, Trajectory

## Creating a Trajectory

You can create trajectories from lists, generators, or file readers:


In [ ]:
# Create some frames
frames = []
for i in range(5):
    frame = mp.Frame()
    frame["atoms"] = mp.Block({"x": [0.0 + i * 0.1], "y": [0.0], "z": [0.0]})
    frame.metadata["time"] = i * 0.1
    frames.append(frame)

# Create trajectory from list
traj = Trajectory(frames)
print(f"Trajectory length: {len(traj)}")

## Iterating Over Frames


In [ ]:
# Iterate through all frames
for i, frame in enumerate(traj):
    time = frame.metadata.get("time", 0.0)
    n_atoms = frame["atoms"].nrows
    print(f"Frame {i}: time={time:.2f}, atoms={n_atoms}")

## Manual Iteration with next()

You can manually iterate through frames using `next()`:


In [ ]:
# Manually get next frames
frame1 = next(traj)
print(f"First frame time: {frame1.metadata.get('time', 0.0)}")

frame2 = next(traj)
print(f"Second frame time: {frame2.metadata.get('time', 0.0)}")

# You can also use try/except to handle StopIteration
try:
    frame3 = next(traj)
    print(f"Third frame time: {frame3.metadata.get('time', 0.0)}")
except StopIteration:
    print("No more frames")

## Accessing Individual Frames

You can also access frames by index:


In [ ]:
# Get a single frame by index
frame0 = traj[0]
print(f"First frame time: {frame0.metadata.get('time', 0.0)}")

# Get last frame (if trajectory has known length)
if traj.has_length():
    frame_last = traj[-1]
    print(f"Last frame time: {frame_last.metadata.get('time', 0.0)}")

## Slicing Trajectories


In [ ]:
# Get first 3 frames
first_three = traj[0:3]
print(f"First 3 frames: {len(first_three)} frames")

# Get every other frame
every_other = traj[::2]
print(f"Every other frame: {len(every_other)} frames")

# Get last 2 frames
last_two = traj[-2:]
print(f"Last 2 frames: {len(last_two)} frames")

## Mapping Functions Over Frames

The `.map()` method applies a function to each frame lazily:


In [ ]:
# Define a function to process frames
def center_frame(frame):
    """Center coordinates at origin."""
    atoms = frame["atoms"]
    xyz = atoms[["x", "y", "z"]]
    center = xyz.mean(axis=0)
    atoms["x"] = atoms["x"] - center[0]
    atoms["y"] = atoms["y"] - center[1]
    atoms["z"] = atoms["z"] - center[2]
    return frame


# Apply to all frames (lazy evaluation)
centered_traj = traj.map(center_frame)
print(f"Centered trajectory: {len(centered_traj)} frames")

# The mapping is lazy - frames are processed on-demand
# You can use next() on the mapped trajectory too
if centered_traj.has_length():
    first_centered = next(centered_traj)
    print("First centered frame processed")

## Loading from Files


In [ ]:
# Example: Load trajectory from files
# Note: These examples require actual trajectory files

# Load trajectory from XYZ file
from pathlib import Path

from molpy.io.trajectory import XYZTrajectoryReader

xyz_file = Path("trajectory.xyz")
if xyz_file.exists():
    reader = XYZTrajectoryReader(xyz_file)
    traj = Trajectory(reader)
    print(f"Loaded trajectory: {len(traj)} frames")
else:
    print("XYZ file not found. Using in-memory trajectory instead.")

# Load from LAMMPS trajectory
from molpy.io.trajectory import LammpsTrajectoryReader

lammps_file = Path("dump.lammpstrj")
if lammps_file.exists():
    reader = LammpsTrajectoryReader(lammps_file)
    traj = Trajectory(reader)
    print(f"Loaded LAMMPS trajectory: {len(traj)} frames")
else:
    print("LAMMPS trajectory file not found.")

## Lazy Loading with Generators


In [ ]:
# Create a generator function
def frame_generator():
    for i in range(10):
        frame = mp.Frame()
        frame["atoms"] = mp.Block({"x": [i], "y": [0.0], "z": [0.0]})
        frame.metadata["time"] = i * 0.1
        yield frame


# Create trajectory from generator
gen_traj = Trajectory(FrameGenerator(frame_generator()))

# Check if length is available
if gen_traj.has_length():
    print(f"Trajectory length: {len(gen_traj)}")
else:
    print("Length not available (generator-based)")

# Still can iterate
count = 0
for frame in gen_traj:
    count += 1
    if count >= 3:
        break
print(f"Iterated through {count} frames")

## Splitting Trajectories


In [ ]:
from molpy.core.trajectory import TrajectorySplitter

# Split by frame interval
splitter = TrajectorySplitter(traj)
segments = splitter.split_frames(interval=2)  # Every 2 frames
print(f"Split into {len(segments)} segments")
for i, seg in enumerate(segments):
    print(f"  Segment {i}: {len(seg)} frames")

## Time-Based Splitting


In [ ]:
# Split by time interval (requires time metadata)
# Note: This requires frames to have "time" in metadata
segments = splitter.split_time(interval=0.5)  # Every 0.5 time units
print(f"Time-based split: {len(segments)} segments")

for i, seg in enumerate(segments):
    if seg.has_length():
        print(f"  Segment {i}: {len(seg)} frames")
    else:
        print(f"  Segment {i}: variable length")

## Analysis Example

Here's a practical example of analyzing a trajectory:


In [ ]:
# Calculate mean position over trajectory
positions = []
for frame in traj:
    atoms = frame["atoms"]
    xyz = atoms[["x", "y", "z"]]
    positions.append(xyz.mean(axis=0))

mean_pos = np.array(positions).mean(axis=0)
print(f"Mean position over trajectory: {mean_pos}")